In [3]:
pip install mplsoccer

  Using cached mplsoccer-1.0.4.tar.gz (57 kB)
  Created wheel for mplsoccer: filename=mplsoccer-1.0.4-py3-none-any.whl size=62952 sha256=c7bb25f54d9bac196c83d07c4e44150d7e6bb8c7b986083c83b9eb45c9702ee2
  Stored in directory: c:\users\parab\appdata\local\pip\cache\wheels\25\b4\d8\16a942bb4a257b70bf229637e1c6578467f6badfa3631a8ebd
Successfully built mplsoccer
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
from copy import deepcopy

import mplsoccer
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
eventsDataLaLiga2019 = pd.read_csv("data/matchwise_events_data.csv", low_memory=False)

In [8]:
seasonMetaDataLaLiga2019 = pd.read_csv("data/season_matchwise_data.csv", low_memory=False)

In [9]:
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 50)

In [10]:
eventsDataLaLiga2019.columns

Index(['match_id', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type.id',
       ...
       'shot.deflected', 'block.save_block', 'foul_committed.penalty',
       'foul_won.penalty', 'goalkeeper.punched_out', 'shot.saved_off_target',
       'goalkeeper.shot_saved_off_target', 'dribble.no_touch',
       'goalkeeper.success_in_play', 'goalkeeper.lost_in_play'],
      dtype='object', length=140)

https://github.com/statsbomb/open-data : data knowledge

In [12]:
eventsDataLaLiga2019.head(2)

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,shot.aerial_won,pass.cut_back,miscontrol.aerial_won,foul_committed.card.id,foul_committed.card.name,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play
0,303421,99d03ac6-bd7b-452a-91be-32e28f96bc1c,1,1,00:00.0,0,0,1,0.0,35,Starting XI,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,442.0,"[{'player': {'id': 6571, 'name': 'Roberto Jimé...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303421,45e9a850-9a07-4266-8c05-66565e7ac83c,2,1,00:00.0,0,0,1,0.0,35,Starting XI,206,Deportivo Alavés,1,Regular Play,217,Barcelona,433.0,"[{'player': {'id': 6590, 'name': 'Norberto Mur...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Event Data

This data contains all the events which occured during the match ie pass , substitution

In [14]:
eventsDataLaLiga2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129059 entries, 0 to 129058
Columns: 140 entries, match_id to goalkeeper.lost_in_play
dtypes: float64(34), int64(10), object(96)
memory usage: 137.9+ MB


In [18]:
eventsDataLaLiga2019.columns.values

array(['match_id', 'id', 'index', 'period', 'timestamp', 'minute',
       'second', 'possession', 'duration', 'type.id', 'type.name',
       'possession_team.id', 'possession_team.name', 'play_pattern.id',
       'play_pattern.name', 'team.id', 'team.name', 'tactics.formation',
       'tactics.lineup', 'related_events', 'off_camera', 'location',
       'player.id', 'player.name', 'position.id', 'position.name',
       'pass.recipient.id', 'pass.recipient.name', 'pass.length',
       'pass.angle', 'pass.height.id', 'pass.height.name',
       'pass.end_location', 'pass.body_part.id', 'pass.body_part.name',
       'pass.type.id', 'pass.type.name', 'carry.end_location',
       'under_pressure', 'dribble.outcome.id', 'dribble.outcome.name',
       'dribble.nutmeg', 'duel.type.id', 'duel.type.name',
       'duel.outcome.id', 'duel.outcome.name', 'pass.outcome.id',
       'pass.outcome.name', 'ball_receipt.outcome.id',
       'ball_receipt.outcome.name', 'counterpress',
       'ball_recovery.

In [19]:
eventsDataLaLiga2019.dtypes

match_id                              int64
id                                   object
index                                 int64
period                                int64
timestamp                            object
minute                                int64
second                                int64
possession                            int64
duration                            float64
type.id                               int64
type.name                            object
possession_team.id                    int64
possession_team.name                 object
play_pattern.id                       int64
play_pattern.name                    object
team.id                               int64
team.name                            object
tactics.formation                   float64
tactics.lineup                       object
related_events                       object
off_camera                           object
location                             object
player.id                       

In [21]:
eventsDataLaLiga2019[['type.id','type.name']].drop_duplicates() #event dictionary for reference

,type.id,type.name
0,35,Starting XI
2,18,Half Start
4,30,Pass
5,42,Ball Receipt*
8,43,Carry
39,17,Pressure
42,14,Dribble
43,4,Duel
87,22,Foul Committed
88,21,Foul Won


In [22]:
eventsPassdf = deepcopy(eventsDataLaLiga2019[eventsDataLaLiga2019['type.id']== 30])

In [23]:
eventsPassdf.shape

(37157, 140)

# Season Data

Data for each match

In [24]:
seasonMetaDataLaLiga2019.columns

Index(['match_id', 'match_date', 'kick_off', 'home_score', 'away_score',
       'match_status', 'match_status_360', 'last_updated', 'last_updated_360',
       'match_week', 'competition.competition_id', 'competition.country_name',
       'competition.competition_name', 'season.season_id',
       'season.season_name', 'home_team.home_team_id',
       'home_team.home_team_name', 'home_team.home_team_gender',
       'home_team.home_team_group', 'home_team.country.id',
       'home_team.country.name', 'home_team.managers',
       'away_team.away_team_id', 'away_team.away_team_name',
       'away_team.away_team_gender', 'away_team.away_team_group',
       'away_team.country.id', 'away_team.country.name', 'away_team.managers',
       'metadata.data_version', 'metadata.shot_fidelity_version',
       'metadata.xy_fidelity_version', 'competition_stage.id',
       'competition_stage.name', 'stadium.id', 'stadium.name',
       'stadium.country.id', 'stadium.country.name', 'referee.id',
       're

In [105]:
seasonMetaDataLaLiga2019.shape

(485, 42)

In [34]:
seasonMetaDataLaLiga2019.head(3)

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,metadata.shot_fidelity_version,metadata.xy_fidelity_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name
0,303421,7/19/2020,00:00.0,0,5,available,unscheduled,2020-07-29T05:00,NaN,38,11,Spain,La Liga,42,2019/2020,206,Deportivo Alavés,male,NaN,214,Spain,"[{'id': 2, 'name': 'Juan Ramón López Muñiz', '...",217,Barcelona,male,NaN,214,Spain,"[{'id': 238, 'name': 'Enrique Setién Solar', '...",1.1.0,2,2.0,1,Regular Season,348.0,Estadio de Mendizorroza,214.0,Spain,207.0,J. Martínez,214.0,Spain
1,303493,6/23/2020,00:00.0,1,0,available,unscheduled,2020-07-29T05:00,NaN,31,11,Spain,La Liga,42,2019/2020,217,Barcelona,male,NaN,214,Spain,"[{'id': 238, 'name': 'Enrique Setién Solar', '...",215,Athletic Bilbao,male,NaN,214,Spain,"[{'id': 767, 'name': 'Gaizka Garitano Aguirre'...",1.1.0,2,2.0,1,Regular Season,342.0,Camp Nou,214.0,Spain,183.0,Jesús Gil,214.0,Spain
2,303516,6/27/2020,00:00.0,2,2,available,unscheduled,2020-07-29T05:00,NaN,32,11,Spain,La Liga,42,2019/2020,209,Celta Vigo,male,NaN,214,Spain,"[{'id': 430, 'name': 'Óscar García Junyent', '...",217,Barcelona,male,NaN,214,Spain,"[{'id': 238, 'name': 'Enrique Setién Solar', '...",1.1.0,2,2.0,1,Regular Season,653.0,Abanca-Balaídos,214.0,Spain,451.0,G. Cuadra,214.0,Spain


In [25]:
seasonCols = ['match_id', 'match_date', 'kick_off', 'home_score', 'away_score',
              'home_team.home_team_id', 'home_team.home_team_name',
              'away_team.away_team_id', 'away_team.away_team_name']

In [33]:
seasonMetaDataLaLiga2019[seasonMetaDataLaLiga2019['match_id']==333734]

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,metadata.shot_fidelity_version,metadata.xy_fidelity_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name


In [39]:
eventsPassdf = pd.merge(eventsPassdf,seasonMetaDataLaLiga2019[seasonCols],
                       how="left",on=['match_id'])

In [40]:
eventsPassdf.shape

(37157, 148)

# Feat Eng.

Here we will check the starting lineup and make table which contains data whether the player started the match or not

In [42]:
eventsDataLaLiga2019.head()

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,shot.aerial_won,pass.cut_back,miscontrol.aerial_won,foul_committed.card.id,foul_committed.card.name,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play
0,303421,99d03ac6-bd7b-452a-91be-32e28f96bc1c,1,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,442.0,"[{'player': {'id': 6571, 'name': 'Roberto Jimé...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303421,45e9a850-9a07-4266-8c05-66565e7ac83c,2,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,217,Barcelona,433.0,"[{'player': {'id': 6590, 'name': 'Norberto Mur...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,303421,715a7900-5e3a-41f6-9870-44dc657ba2bc,3,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,217,Barcelona,NaN,NaN,['9df4334c-c6e4-4457-97f2-8948375695c7'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,303421,9df4334c-c6e4-4457-97f2-8948375695c7,4,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['715a7900-5e3a-41f6-9870-44dc657ba2bc'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,303421,1a45cc07-8721-46a1-81eb-ac83d3e44b6d,5,1,00:00.6,0,0,2,1.235451,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,['087f53e9-22aa-48fb-b8bf-0a294b8c3262'],True,"[60.0, 40.0]",5246.0,Luis Alberto Suárez Díaz,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303707)
                    & (eventsDataLaLiga2019['type.id'] == 35)]['tactics.lineup'] 

87378    [{'player': {'id': 16309, 'name': 'Joel Robles...
87379    [{'player': {'id': 20055, 'name': 'Marc-André ...
Name: tactics.lineup, dtype: object

In [51]:
eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303707)
                    & (eventsDataLaLiga2019['type.id'] == 35)]['tactics.lineup'].iloc[0]

"[{'player': {'id': 16309, 'name': 'Joel Robles Blázquez'}, 'position': {'id': 1, 'name': 'Goalkeeper'}, 'jersey_number': 1}, {'player': {'id': 23522, 'name': 'Emerson Aparecido Leite de Souza Junior'}, 'position': {'id': 2, 'name': 'Right Back'}, 'jersey_number': 22}, {'player': {'id': 6648, 'name': 'Aïssa Mandi'}, 'position': {'id': 3, 'name': 'Right Center Back'}, 'jersey_number': 23}, {'player': {'id': 7068, 'name': 'Marc Bartra Aregall'}, 'position': {'id': 5, 'name': 'Left Center Back'}, 'jersey_number': 5}, {'player': {'id': 11538, 'name': 'Alejandro Moreno Lopera'}, 'position': {'id': 6, 'name': 'Left Back'}, 'jersey_number': 15}, {'player': {'id': 26404, 'name': 'Guido Rodríguez'}, 'position': {'id': 10, 'name': 'Center Defensive Midfield'}, 'jersey_number': 21}, {'player': {'id': 13599, 'name': 'Carles Aleña Castillo'}, 'position': {'id': 12, 'name': 'Right Midfield'}, 'jersey_number': 24}, {'player': {'id': 6673, 'name': 'Sergio Canales Madrazo'}, 'position': {'id': 13, 'nam

In [52]:
from ast import literal_eval

In [53]:
eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303707)
                    & (eventsDataLaLiga2019['type.id'] == 35)]['tactics.lineup'].apply(literal_eval).iloc[0]

[{'player': {'id': 16309, 'name': 'Joel Robles Blázquez'},
  'position': {'id': 1, 'name': 'Goalkeeper'},
  'jersey_number': 1},
 {'player': {'id': 23522, 'name': 'Emerson Aparecido Leite de Souza Junior'},
  'position': {'id': 2, 'name': 'Right Back'},
  'jersey_number': 22},
 {'player': {'id': 6648, 'name': 'Aïssa Mandi'},
  'position': {'id': 3, 'name': 'Right Center Back'},
  'jersey_number': 23},
 {'player': {'id': 7068, 'name': 'Marc Bartra Aregall'},
  'position': {'id': 5, 'name': 'Left Center Back'},
  'jersey_number': 5},
 {'player': {'id': 11538, 'name': 'Alejandro Moreno Lopera'},
  'position': {'id': 6, 'name': 'Left Back'},
  'jersey_number': 15},
 {'player': {'id': 26404, 'name': 'Guido Rodríguez'},
  'position': {'id': 10, 'name': 'Center Defensive Midfield'},
  'jersey_number': 21},
 {'player': {'id': 13599, 'name': 'Carles Aleña Castillo'},
  'position': {'id': 12, 'name': 'Right Midfield'},
  'jersey_number': 24},
 {'player': {'id': 6673, 'name': 'Sergio Canales Madr

In [60]:
pd.json_normalize(eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303707)
                    & (eventsDataLaLiga2019['type.id'] == 35)]['tactics.lineup'].apply(literal_eval).iloc[1])

,jersey_number,player.id,player.name,position.id,position.name
0,1,20055,Marc-André ter Stegen,1,Goalkeeper
1,2,6374,Nélson Cabral Semedo,2,Right Back
2,23,5492,Samuel Yves Umtiti,3,Right Center Back
3,15,6826,Clément Lenglet,5,Left Center Back
4,24,17304,Héctor Junior Firpo Adames,6,Left Back
5,5,5203,Sergio Busquets i Burgos,10,Center Defensive Midfield
6,20,6379,Sergi Roberto Carnicer,13,Right Center Midfield
7,21,8118,Frenkie de Jong,15,Left Center Midfield
8,22,8206,Arturo Erasmo Vidal Pardo,19,Center Attacking Midfield
9,10,5503,Lionel Andrés Messi Cuccittini,22,Right Center Forward


In [55]:
df = pd.json_normalize(eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303707)
                    & (eventsDataLaLiga2019['type.id'] == 35)]['tactics.lineup'].apply(literal_eval).iloc[0])

In [56]:
df

,jersey_number,player.id,player.name,position.id,position.name
0,1,16309,Joel Robles Blázquez,1,Goalkeeper
1,22,23522,Emerson Aparecido Leite de Souza Junior,2,Right Back
2,23,6648,Aïssa Mandi,3,Right Center Back
3,5,7068,Marc Bartra Aregall,5,Left Center Back
4,15,11538,Alejandro Moreno Lopera,6,Left Back
5,21,26404,Guido Rodríguez,10,Center Defensive Midfield
6,24,13599,Carles Aleña Castillo,12,Right Midfield
7,10,6673,Sergio Canales Madrazo,13,Right Center Midfield
8,14,5214,William Silva de Carvalho,15,Left Center Midfield
9,8,2948,Nabil Fekir,16,Left Midfield


In [57]:
df['started'] = "yes" #putting yes if person in starting lineup ie the player started

In [58]:
df.insert(0,"match_id",303707)

In [59]:
df

,match_id,jersey_number,player.id,player.name,position.id,position.name,started
0,303707,1,16309,Joel Robles Blázquez,1,Goalkeeper,yes
1,303707,22,23522,Emerson Aparecido Leite de Souza Junior,2,Right Back,yes
2,303707,23,6648,Aïssa Mandi,3,Right Center Back,yes
3,303707,5,7068,Marc Bartra Aregall,5,Left Center Back,yes
4,303707,15,11538,Alejandro Moreno Lopera,6,Left Back,yes
5,303707,21,26404,Guido Rodríguez,10,Center Defensive Midfield,yes
6,303707,24,13599,Carles Aleña Castillo,12,Right Midfield,yes
7,303707,10,6673,Sergio Canales Madrazo,13,Right Center Midfield,yes
8,303707,14,5214,William Silva de Carvalho,15,Left Center Midfield,yes
9,303707,8,2948,Nabil Fekir,16,Left Midfield,yes


In [67]:
#looping for each match id and adding data for starting lineup for each match in a dataframe
finalLineUpdf  = pd.DataFrame()
for m_id in eventsDataLaLiga2019['match_id'].unique():
    print(m_id,"\n")
    match_df = eventsDataLaLiga2019[(eventsDataLaLiga2019["match_id"] == m_id)
                                   & (eventsDataLaLiga2019["type.id"] == 35)]["tactics.lineup"].apply(literal_eval)
    df1 = pd.json_normalize(match_df.iloc[0])
    df2 = pd.json_normalize(match_df.iloc[1])
    df = df1.append(df2)
    df.insert(0,"match_id",m_id)
    df['started'] = "Yes"
    
    finalLineUpdf = finalLineUpdf.append(df)

    

303421 

303493 

303516 

303680 

303532 

303400 

303634 

303479 

303615 

303696 

303664 

303596 

303487 

303600 

303548 

303473 

303610 

303652 

303430 

303674 

303470 

303700 

303707 

303666 

303725 

303504 

303715 

303377 

303524 

303451 

303517 

303682 

303731 



In [68]:
finalLineUpdf.head()

,match_id,jersey_number,player.id,player.name,position.id,position.name,started
0,303421,13,6571,Roberto Jiménez Gago,1,Goalkeeper,Yes
1,303421,21,6618,Martín Aguirregabiria Padilla,2,Right Back,Yes
2,303421,5,6615,Víctor Laguardia Cisneros,3,Right Center Back,Yes
3,303421,6,23263,Lisandro Magallán,5,Left Center Back,Yes
4,303421,17,6935,Adrián Marín Gómez,6,Left Back,Yes


In [69]:
finalLineUpdf.columns

Index(['match_id', 'jersey_number', 'player.id', 'player.name', 'position.id',
       'position.name', 'started'],
      dtype='object')

In [70]:
#merge the data with original dataframe based on match
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,
                               finalLineUpdf[['match_id','player.id','jersey_number','started']]
                               ,how="left",on=['match_id','player.id'])

In [75]:
eventsDataLaLiga2019[(eventsDataLaLiga2019['match_id'] == 303421) & (eventsDataLaLiga2019['player.id'] == 6571)]

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,miscontrol.aerial_won,foul_committed.card.id,foul_committed.card.name,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play,jersey_number,started
182,303421,fe3e7740-c1b0-424e-9b4c-6be1a8e7252e,183,1,02:53.0,2,52,3,0.000000,23,Goal Keeper,217,Barcelona,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,['e9691e6d-3ccb-49c1-8598-7796361df39f'],NaN,"[1.5, 41.3]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
328,303421,3245b67e-13bd-4b0a-9e62-dea7ccb6b7d3,329,1,05:47.4,5,47,9,0.000000,2,Ball Recovery,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['b4a84b44-5d6e-4676-8495-5612cb33891c'],NaN,"[15.7, 42.2]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
329,303421,25e9cc0f-a443-46ef-8cd8-ecb192d62feb,330,1,05:47.4,5,47,9,1.676789,43,Carry,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,"['3245b67e-13bd-4b0a-9e62-dea7ccb6b7d3', '85f9...",NaN,"[15.7, 42.2]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
330,303421,85f9ac6c-67a4-43df-95da-38e6b87d68c3,331,1,05:49.1,5,49,9,1.234000,30,Pass,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['c3edd4f3-27f0-4cd0-8cf2-e96946ad9abe'],NaN,"[15.4, 41.0]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
346,303421,035f5d84-962c-4655-865b-3b07bd0177b5,347,1,06:01.1,6,1,9,NaN,42,Ball Receipt*,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['574cb166-10ea-4f05-9e99-a66d8ccb9f94'],NaN,"[2.9, 32.5]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
347,303421,861356da-2051-4c7f-a002-d14ea7714318,348,1,06:01.1,6,1,9,1.370247,43,Carry,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,"['035f5d84-962c-4655-865b-3b07bd0177b5', 'cd1a...",NaN,"[2.9, 32.5]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
348,303421,cd1a9330-34a5-4205-b6cc-46b1f13c6428,349,1,06:02.4,6,2,9,3.236686,30,Pass,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['5d25ebbe-b46a-44e7-8919-3dc005d66dee'],NaN,"[3.2, 36.3]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
416,303421,46489e19-c338-419f-b84b-de6eb73a98f9,417,1,07:26.4,7,26,11,0.000000,2,Ball Recovery,217,Barcelona,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,NaN,NaN,"[3.2, 41.0]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
417,303421,fdd48cb4-a109-4c1c-95be-39f1b8f2ee33,418,1,07:41.4,7,41,11,3.454497,30,Pass,217,Barcelona,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,['24c9c3ea-b083-4209-ab55-dd3172e88dad'],True,"[9.4, 36.3]",6571.0,Roberto Jiménez Gago,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,Yes
472,303421,4ce4e91f-a468-4ce1-a96b-9cb988ada664,473,1,09:01.8,9,1,14,0.000000,23

### `minsPlayed` Column

Finding mins played for each player in a match and doing that for all matches

In [76]:
minsPlayeddf = eventsDataLaLiga2019.groupby(['match_id',"player.id"]).agg(
{'player.name':'first',"team.name":"first","minute":{"min","max"}}).reset_index()

In [77]:
minsPlayeddf.head() # for match id 303377 robert kenedy made app. at 53 min ie got subbed on at 53 min 

match_id player.id                        player.name  team.name minute    
                                                  first      first    min max
0   303377    3666.0  Robert Kenedy Nunes do Nascimento     Getafe     53  92
1   303377    3991.0                   Allan Romeo Nyom     Getafe      1  92
2   303377    5203.0           Sergio Busquets i Burgos  Barcelona      0  93
3   303377    5211.0                   Jordi Alba Ramos  Barcelona      1  21
4   303377    5213.0              Gerard Piqué Bernabéu  Barcelona      0  91

In [78]:
eventsDataLaLiga2019[eventsDataLaLiga2019['substitution.outcome.id'].notnull()]\
[["match_id", "player.name", 'substitution.replacement.id', 'substitution.replacement.name', "minute"]] #checking substitutions made in each match

,match_id,player.name,substitution.replacement.id,substitution.replacement.name,minute
1936,303421,Lucas Pérez Martínez,6565.0,Pere Pons Riera,45
1937,303421,Oliver Burke,24049.0,Luis Jesús Rioja González,45
1938,303421,Víctor Laguardia Cisneros,30419.0,Alberto Rodríguez Baró,45
2182,303421,Clément Lenglet,6374.0,Nélson Cabral Semedo,49
2522,303421,Sergio Busquets i Burgos,8118.0,Frenkie de Jong,58
...,...,...,...,...,...
128104,303731,Takashi Inui,8555.0,Pablo Ezequiel De Blasis,65
128105,303731,Sergio Enrich Ametller,23808.0,Enrique González Casín,65
128189,303731,Gerard Piqué Bernabéu,5492.0,Samuel Yves Umtiti,67
128331,303731,Antoine Griezmann,4447.0,Martin Braithwaite Christensen,71


In [83]:
subData = eventsDataLaLiga2019[['match_id',"substitution.replacement.id","minute"]]\
.dropna(subset=['substitution.replacement.id'])

In [84]:
subData.rename(columns={"minute":"minMinute",
                       "substitution.replacement.id":"subID"},inplace=True)

In [85]:
subData.head()

,match_id,subID,minMinute
1936,303421,6565.0,45
1937,303421,24049.0,45
1938,303421,30419.0,45
2182,303421,6374.0,49
2522,303421,8118.0,58


In [86]:
pd.merge(eventsDataLaLiga2019,subData,how="left",
        left_on=['match_id','player.id'],right_on=['match_id','subID']) #merge subbed on time with og data

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,foul_committed.card.name,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play,jersey_number,started,subID,minMinute
0,303421,99d03ac6-bd7b-452a-91be-32e28f96bc1c,1,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,442.0,"[{'player': {'id': 6571, 'name': 'Roberto Jimé...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303421,45e9a850-9a07-4266-8c05-66565e7ac83c,2,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,217,Barcelona,433.0,"[{'player': {'id': 6590, 'name': 'Norberto Mur...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,303421,715a7900-5e3a-41f6-9870-44dc657ba2bc,3,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,217,Barcelona,NaN,NaN,['9df4334c-c6e4-4457-97f2-8948375695c7'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,303421,9df4334c-c6e4-4457-97f2-8948375695c7,4,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['715a7900-5e3a-41f6-9870-44dc657ba2bc'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,303421,1a45cc07-8721-46a1-81eb-ac83d3e44b6d,5,1,00:00.6,0,0,2,1.235451,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,['087f53e9-22aa-48fb-b8bf-0a294b8c3262'],True,"[60.0, 40.0]",5246.0,Luis Alberto Suárez Díaz,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129054,303731,bbcfd05f-b1b3-4c8c-8164-131f66073442,3686,2,46:56.3,91,56,184,0.000000,10,Interception,217,Barcelona,1,Regular Play,322,Eibar,NaN,NaN,['9bbe3056-7489-489d-bfbb-0bb98730bb66'],NaN,"[40.5, 53.8]",7105.0,Fabián Ariel Orellana Valenzuela,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,Yes,NaN,NaN
129055,303731,b83338c5-c933-4b5c-b7c5-74a59a1a105b,3687,2,46:57.6,91,57,184,0.782868,30,Pass,217,Barcelona,1,Regular Play,322,Eibar,NaN,NaN,['cb770319-a70c-4798-8933-fbda94906281'],NaN,"[33.9, 50.0]",23803.0,Esteban Rodrigo Burgos,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,Yes,NaN,NaN
129056,303731,cb770319-a70c-4798-8933-fbda94906281,3688,2,46:58.4,91,58,184,NaN,42,Ball Receipt*,217,Barcelona,1,Regular Play,322,Eibar,NaN,NaN,['b83338c5-c933-4b5c-b7c5-74a59a1a105b'],NaN,"[38.6, 57.7]",6712.0,Gonzalo Escalante,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,Yes,NaN,NaN
129057,303731,57bc9f35-41c3-40bb-848e-815d4086a3f1,3689,2,46:59.4,91,59,184,0.000000,34,Half End,217,Barcelona,1,Regular Play,217,Barcelona,NaN,NaN,['e14c793b-e5a0-4b19-9159-62a34a881d80'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,subData,how="left",
        left_on=['match_id','player.id'],right_on=['match_id','subID'])

In [90]:
eventsDataLaLiga2019[eventsDataLaLiga2019['match_id'] == 303421]

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,foul_committed.card.name,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play,jersey_number,started,subID,minMinute
0,303421,99d03ac6-bd7b-452a-91be-32e28f96bc1c,1,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,442.0,"[{'player': {'id': 6571, 'name': 'Roberto Jimé...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303421,45e9a850-9a07-4266-8c05-66565e7ac83c,2,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,217,Barcelona,433.0,"[{'player': {'id': 6590, 'name': 'Norberto Mur...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,303421,715a7900-5e3a-41f6-9870-44dc657ba2bc,3,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,217,Barcelona,NaN,NaN,['9df4334c-c6e4-4457-97f2-8948375695c7'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,303421,9df4334c-c6e4-4457-97f2-8948375695c7,4,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['715a7900-5e3a-41f6-9870-44dc657ba2bc'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,303421,1a45cc07-8721-46a1-81eb-ac83d3e44b6d,5,1,00:00.6,0,0,2,1.235451,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,['087f53e9-22aa-48fb-b8bf-0a294b8c3262'],True,"[60.0, 40.0]",5246.0,Luis Alberto Suárez Díaz,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3974,303421,33d07e15-a965-495c-abde-880cd5ba588e,3975,2,49:03.4,94,3,166,1.126102,30,Pass,206,Deportivo Alavés,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,['ab7237b9-fcfa-4ec0-9a0c-57887bb1e5dc'],NaN,"[74.4, 57.3]",10318.0,Ljubomir Fejsa,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10318.0,58.0
3975,303421,ab7237b9-fcfa-4ec0-9a0c-57887bb1e5dc,3976,2,49:04.5,94,4,166,NaN,42,Ball Receipt*,206,Deportivo Alavés,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,['33d07e15-a965-495c-abde-880cd5ba588e'],NaN,"[56.6, 57.5]",30419.0,Alberto Rodríguez Baró,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30419.0,45.0
3976,303421,82cb752a-e544-41c9-b5a7-41b0fe0f8e82,3977,2,49:04.6,94,4,166,0.000000,26,Player On,206,Deportivo Alavés,3,From Free Kick,206,Deportivo Alavés,NaN,NaN,['9ff7b74e-f49c-4261-80fa-e8b081108300'],NaN,NaN,6618.0,Martín Aguirregabiria Padilla,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,Yes,NaN,NaN
3977,303421,fe229cb5-c8f5-40ed-a162-f79fbc5c75d4,3978,2,49:05.6,94,5,166,0.000000,34,Half End,206,Deportivo Alavés,3,From Free Kick,217,Barcelona,NaN,NaN,['e71ea791-fe33-4537-bc26-591999989d58'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
eventsDataLaLiga2019.dropna(subset=["started"])["minMinute"].unique() 

array([nan])

In [92]:
eventsDataLaLiga2019["minMinute"] = np.where(eventsDataLaLiga2019['started']=="Yes",
                                            0,eventsDataLaLiga2019['minMinute'])
#create new column for min minute. If player started but 0 else put min minute value which is the min player got subbed on

In [95]:
eventsDataLaLiga2019.groupby(['match_id','player.id']).agg(
{'player.name':"first",'team.name':"first",
"minMinute":"first","minute":"max"}).reset_index()

,match_id,player.id,player.name,team.name,minMinute,minute
0,303377,3666.0,Robert Kenedy Nunes do Nascimento,Getafe,51.0,92
1,303377,3991.0,Allan Romeo Nyom,Getafe,0.0,92
2,303377,5203.0,Sergio Busquets i Burgos,Barcelona,0.0,93
3,303377,5211.0,Jordi Alba Ramos,Barcelona,0.0,21
4,303377,5213.0,Gerard Piqué Bernabéu,Barcelona,0.0,91
...,...,...,...,...,...,...
947,303731,17304.0,Héctor Junior Firpo Adames,Barcelona,0.0,91
948,303731,20055.0,Marc-André ter Stegen,Barcelona,0.0,91
949,303731,23803.0,Esteban Rodrigo Burgos,Eibar,0.0,91
950,303731,23808.0,Enrique González Casín,Eibar,65.0,91


In [96]:
minsPlayeddf = eventsDataLaLiga2019.groupby(['match_id','player.id']).agg(
{'player.name':"first",'team.name':"first",
"minMinute":"first","minute":"max"}).reset_index()

In [99]:
minsPlayeddf['minsPlayed'] = minsPlayeddf['minute'].subtract(minsPlayeddf['minMinute'])

In [100]:
eventsDataLaLiga2019 = pd.merge(eventsDataLaLiga2019,
                       minsPlayeddf[["match_id", "player.id", "minsPlayed"]],
                       how="left",
                       on=["match_id", "player.id"])
eventsDataLaLiga2019.head()

,match_id,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,off_camera,location,player.id,player.name,position.id,...,pass.inswinging,clearance.aerial_won,clearance.other,shot.open_goal,pass.straight,shot.saved_to_post,goalkeeper.shot_saved_to_post,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,block.save_block,foul_committed.penalty,foul_won.penalty,goalkeeper.punched_out,shot.saved_off_target,goalkeeper.shot_saved_off_target,dribble.no_touch,goalkeeper.success_in_play,goalkeeper.lost_in_play,jersey_number,started,subID,minMinute,minsPlayed
0,303421,99d03ac6-bd7b-452a-91be-32e28f96bc1c,1,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,442.0,"[{'player': {'id': 6571, 'name': 'Roberto Jimé...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,303421,45e9a850-9a07-4266-8c05-66565e7ac83c,2,1,00:00.0,0,0,1,0.000000,35,Starting XI,206,Deportivo Alavés,1,Regular Play,217,Barcelona,433.0,"[{'player': {'id': 6590, 'name': 'Norberto Mur...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,303421,715a7900-5e3a-41f6-9870-44dc657ba2bc,3,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,217,Barcelona,NaN,NaN,['9df4334c-c6e4-4457-97f2-8948375695c7'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,303421,9df4334c-c6e4-4457-97f2-8948375695c7,4,1,00:00.0,0,0,1,0.000000,18,Half Start,206,Deportivo Alavés,1,Regular Play,206,Deportivo Alavés,NaN,NaN,['715a7900-5e3a-41f6-9870-44dc657ba2bc'],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,303421,1a45cc07-8721-46a1-81eb-ac83d3e44b6d,5,1,00:00.6,0,0,2,1.235451,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,['087f53e9-22aa-48fb-b8bf-0a294b8c3262'],True,"[60.0, 40.0]",5246.0,Luis Alberto Suárez Díaz,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,Yes,NaN,0.0,92.0


In [101]:
eventsDataLaLiga2019.to_csv("data/matchwise_events_data_updated.csv", index=False)